## Treino

Depois de uma otimização de hiperparâmetros intensiva consegui o melhor modelo para a inferência de probabilidade de Churn. Os meus critérios para escolha do algoritmo certo foram: desempenho e simplicidade.

Acredito que os dois algoritmos mais simples para lidar com classes desbalanceadas são: RandomForestClassifier e DecisionTreeClassifier. No começo, esses dados me deram bastante trabalho deviddo a natureza do problema. Churn é sempre assim. Desbalanceado demais. Eu optei por deixar de fora o DecisionTreeClassifier pois o RandomForestClassifier já não estava coseguindo lidar tão facilmente e inclui XGBoost na experimentação pois teria acesso a Learning Rate.

No final das contas, depois de muitas horas de otimização de hiperpoarâmetros, o RandomForestClassifier se mostrou o melhor modelo, conseguindo identificar as duas classes com tranquilidade.

Este notebook será somente o treinamento do modelo, com os hiperparâmetros que o RandomizedSearchCV + BayesSearchCV encontraram juntos para formar esse excelente modelo.

In [12]:
import pandas as pd

In [ ]:
PATH_FILE = "../../data/num.csv"

In [6]:
df = pd.read_csv(PATH_FILE)

X = df.drop(columns=['churn'])
y = df['churn']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        random_state=42,
        stratify=y
)

In [7]:
print(f"Train: {X_train.shape}\nTest: {X_test.shape}")

Train: (428239, 5)
Test: (107060, 5)


In [8]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
        bootstrap=True,
        class_weight="balanced",
        max_depth=30,
        min_samples_leaf=2,
        min_samples_split=5,
        n_estimators=100,
        random_state=42
)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=30,
                       min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [9]:
from sklearn.metrics import (
      accuracy_score,
      recall_score,
      precision_score,
      f1_score,
      roc_auc_score,
      confusion_matrix,
      classification_report
)

y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
rocauc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [10]:
print("1. Classification Report\n", cr)
print("\n2. Confusion Matrix\n", cm)
print("\n3. ROC AUC Score: ", rocauc)
print("\n4. Accuracy Score: ", acc)
print("\n5. Precision Score: ", precision)
print("\n6. Recall Score: ", recall)
print("\n7. F1 Score: ", f1)

1. Classification Report
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     28687
           1       0.99      1.00      0.99     78373

    accuracy                           0.99    107060
   macro avg       0.99      0.99      0.99    107060
weighted avg       0.99      0.99      0.99    107060


2. Confusion Matrix
 [[27959   728]
 [  329 78044]]

3. ROC AUC Score:  0.9992430812583394

4. Accuracy Score:  0.9901270315710816

5. Precision Score:  0.9907581374092317

6. Recall Score:  0.995802125732076

7. F1 Score:  0.993273728085526
